In [2]:
import pickle
import datetime as dt
import pandas as pd

from st_order_tools.order_tools.controller import Controller, StatusController, OrderController
from private import api_key, api_secret, acc_no

In [3]:
controller = Controller(api_key, api_secret, acc_no)
controller.set_access_token()

In [4]:
status_controller = StatusController(controller)

In [5]:
balance_df = status_controller.load_balance()
balance_df

,cash,stock,total
0,47222.0,51380,98602


In [6]:
position_df = status_controller.load_position()
position_df

,stock_code,stock_nm,buying_price,current_price,quantity,profit_pct
0,008110,대동전자,6890.0000,6740,5,-2.177%
1,011200,HMM,18860.0000,17670,1,-6.31%


In [7]:
with open("./data/buying_stock_price.pkl", "rb") as f:
    buying_stock_price = pickle.load(f)

cash = balance_df["cash"].iloc[0]
buying_orders = [
    (stock_code, int((cash // len(buying_stock_price)) // price))
    for (stock_code, price) in buying_stock_price
]

In [9]:
order_controller = OrderController(controller)

buying_resps = list()
for buying_order in buying_orders:
    resp = order_controller.make_market_buy_order(*buying_order)
    buying_resps.append(resp)

In [10]:
_buying_resps_df = pd.DataFrame(buying_resps)
buying_resps_df = pd.concat(
    [
        _buying_resps_df["meta"].apply(pd.Series),
        _buying_resps_df.loc[:, ["msg1"]],
    ],
    axis=1,
)
buying_resps_df

,stock_code,quantity,msg1
0,008110,3,주문 전송 완료 되었습니다.
1,002920,7,주문 전송 완료 되었습니다.


In [11]:
lower_limit = -10
upper_limit = 30


def filter_upper_limit(position_df, limit):
    filtered_position_df = position_df[
        position_df["profit_pct"].apply(lambda x: limit < float(x.rstrip("%")))
    ]
    return filtered_position_df


def filter_lower_limit(position_df, limit):
    filtered_position_df = position_df[
        position_df["profit_pct"].apply(lambda x: float(x.rstrip("%")) < limit)
    ]
    return filtered_position_df


selling_position_df = pd.concat(
    [filter_lower_limit(position_df, -10), filter_upper_limit(position_df, 30)], axis=0
)
selling_orders = list(
    selling_position_df.loc[:, ["stock_code", "quantity"]].itertuples(index=False, name=None)
)
selling_orders

[]

In [12]:
if selling_orders:
    selling_resps = list()
    for selling_order in selling_orders:
        resp = order_controller.make_market_sell_order(*selling_order)
        selling_resps.append(resp)
    _selling_resps_df = pd.DataFrame(selling_resps)
    selling_resps_df = pd.concat(
        [
            _selling_resps_df["meta"].apply(pd.Series),
            _selling_resps_df.loc[:, ["msg1"]],
        ],
        axis=1,
    )
    selling_resps_df